In [ ]:
import email
import imaplib
import os
import re
from email.header import decode_header

from dotenv import load_dotenv

load_dotenv()

True

In [6]:
mail = imaplib.IMAP4_SSL(os.getenv("IMAP_SERVER"))
mail.login((os.getenv("EMAIL_ACCOUNT")), os.getenv("PASSWORD"))
mail.select("inbox")

('OK', [b'8'])

In [7]:
status, messages = mail.search(None, "ALL")
for num in messages[0].split():
    status, data = mail.fetch(num, "(RFC822)")
    raw_email = data[0][1]
    msg = email.message_from_bytes(raw_email)
    print("From:", msg["From"])
    print("Subject:", msg["Subject"])
    print("="*50)

From: support@docsmedicalbilling.com
Subject: Welcome to Docs Medical Billing
From: welcome@zoho.com
Subject: Access from anywhere, anytime!
From: systemgenerated@mailer.zohocliq.com
Subject: Cliq: Knock, Knock! You've got an unread chat.
From: huggingface <website@huggingface.co>
Subject: [Hugging Face] Click this link to confirm your email address
From: Human Resources <hrd@docsmedicalbilling.com>
Subject: Segregation of Monthly Fuel Entitlement
From: Human Resources <hrd@docsmedicalbilling.com>
Subject: Strict Adherence to Dress Code Policy
From: Human Resources <hrd@docsmedicalbilling.com>
Subject: Display of Employee ID Cards & Cooperation with Security Guard
From: Human Resources <hrd@docsmedicalbilling.com>
Subject: Dress Code for 14th August Celebration


In [ ]:
def clean_html(raw_html):
    """Remove HTML tags & extra spaces."""
    clean_text = re.sub(r'<[^>]+>', '', raw_html)
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()
    return clean_text

def connect_mailbox():
    """Connect to Zoho IMAP inbox."""
    mail = imaplib.IMAP4_SSL(os.getenv("IMAP_SERVER"))
    mail.login(os.getenv("EMAIL_ACCOUNT"), os.getenv("PASSWORD"))
    mail.select("inbox")
    return mail

def fetch_latest_emails(n=10):
    """Fetch latest N emails as list of strings."""
    try:
        mail = connect_mailbox()
        status, email_ids = mail.search(None, "ALL")
    except imaplib.IMAP4.abort:
        print("[INFO] Reconnecting due to server logout...")
        mail = connect_mailbox()
        status, email_ids = mail.search(None, "ALL")

    email_ids = email_ids[0].split()[-n:][::-1]  # latest first

    emails_list = []

    for num in email_ids:
        try:
            status, msg_data = mail.fetch(num, "(RFC822)")
            if status != "OK":
                continue

            msg = email.message_from_bytes(msg_data[0][1])

            # Decode subject
            subject, encoding = decode_header(msg["Subject"])[0]
            if isinstance(subject, bytes):
                subject = subject.decode(encoding or "utf-8", errors="ignore")

            # From
            from_ = msg.get("From", "")

            # Date
            date_ = msg.get("Date", "")

            # Body
            body = ""
            if msg.is_multipart():
                for part in msg.walk():
                    content_type = part.get_content_type()
                    content_dispo = str(part.get("Content-Disposition"))
                    if content_type == "text/plain" and "attachment" not in content_dispo:
                        body += part.get_payload(decode=True).decode(errors="ignore")
                    elif content_type == "text/html" and not body:
                        html_content = part.get_payload(decode=True).decode(errors="ignore")
                        body += clean_html(html_content)
            else:
                if msg.get_content_type() == "text/plain":
                    body = msg.get_payload(decode=True).decode(errors="ignore")
                elif msg.get_content_type() == "text/html":
                    html_content = msg.get_payload(decode=True).decode(errors="ignore")
                    body = clean_html(html_content)

            # Combine into one string
            full_email = f"Subject: {subject}\nFrom: {from_}\nDate: {date_}\n\nBody:\n{body}"
            emails_list.append(full_email)

        except Exception as e:
            print(f"[ERROR] Could not process email {num}: {e}")

    mail.close()
    mail.logout()
    return emails_list


latest_emails = fetch_latest_emails(n=5)  # Get latest 5 emails
for i, em in enumerate(latest_emails, 1):
    print(f"--- EMAIL {i} ---\n{em}\n{'-'*50}")

--- EMAIL 1 ---
Subject: Dress Code for 14th August Celebration
From: Human Resources <hrd@docsmedicalbilling.com>
Date: Tue, 12 Aug 2025 01:46:05 +0400

Body:
BCC: All ConcernsDear Team,As part of our 14th August celebration on Wednesday, we kindly request all employees to follow the specified theme attire to make the event more vibrant and unified.Male employees:&nbsp;Please wear a white salwar kameez or a white shirt along with cotton pants/jeans.Female employees:&nbsp;Please follow the white and green attire theme.This dress code is mandatory&nbsp;for everyone, as we will be having a group photo shoot on Wednesday to commemorate the celebration.Your cooperation is highly appreciated to make this event memorable.Thank you and looking forward to your enthusiastic participation!Regards, &nbsp; &nbsp; &nbsp;&nbsp;Human Resources Department&nbsp;Docs Medical Billing LLC.&nbsp;email: hrd@docsmedicalbilling.comphone: +1 (559) 330-1413Website: www.docsmedicalbilling.com&nbsp;WARNING: CONFI

In [21]:
for e in latest_emails:
    print(e)
    print("="*50)

Subject: Dress Code for 14th August Celebration
From: Human Resources <hrd@docsmedicalbilling.com>
Date: Tue, 12 Aug 2025 01:46:05 +0400

Body:
BCC: All ConcernsDear Team,As part of our 14th August celebration on Wednesday, we kindly request all employees to follow the specified theme attire to make the event more vibrant and unified.Male employees:&nbsp;Please wear a white salwar kameez or a white shirt along with cotton pants/jeans.Female employees:&nbsp;Please follow the white and green attire theme.This dress code is mandatory&nbsp;for everyone, as we will be having a group photo shoot on Wednesday to commemorate the celebration.Your cooperation is highly appreciated to make this event memorable.Thank you and looking forward to your enthusiastic participation!Regards, &nbsp; &nbsp; &nbsp;&nbsp;Human Resources Department&nbsp;Docs Medical Billing LLC.&nbsp;email: hrd@docsmedicalbilling.comphone: +1 (559) 330-1413Website: www.docsmedicalbilling.com&nbsp;WARNING: CONFIDENTIALITY NOTIC

In [22]:
import json
with open("emails.json", "w", encoding="utf-8") as f:
    json.dump(latest_emails, f, ensure_ascii=False, indent=2)